In [120]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity




In [121]:
df=pd.read_csv('spotify_millsongdata.csv')

In [122]:
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [123]:
df=df.sample(20000).drop('link', axis=1).reset_index(drop=True)


In [185]:
pp=df.copy()

In [186]:
pp['text'] = pp['text'].apply(lambda x: str(x).lower().replace(r'^\w\s',' ').replace(r'\n',' ',))

NOW that we have assigned the link to another column we can drop them


In [126]:
df.shape

(20000, 3)

In [127]:
artist_list=df.groupby('artist').count()

Now we are having 643 artist list in our dataset


In [128]:
artist_list.count()

song    635
text    635
dtype: int64

In [129]:
df['song']=df['song'].str.lower()

In [130]:

p=df['text'].str.count('\n').sum()

In [131]:
df['text'] = df['text'].apply(lambda x: str(x).lower().replace(r'^\w\s',' ').replace(r'\n',' ',))


In [132]:
df.columns

Index(['artist', 'song', 'text'], dtype='object')

In [133]:

# import nltk
# it is a nlp library , it is used to process the text data
# from nltk.stem.porter import PorterStemmer
stemmer=PorterStemmer()

In this the similar words which are having same meaning but differ in spelling are changed into one common word


In [134]:
def token(text):
    token=nltk.word_tokenize(text)
    a=[stemmer.stem(word) for word in token]
    return " ".join(a)
    

In [135]:
# # import nltk
# # nltk.download('punkt')
# token(df['text'][0])
# token('hello world beautiful beauty')

In [136]:
df['text'].apply(lambda x:token(x))

0        special guest vocalist , mike 's dad : bob mcc...
1        anoth sleepless night i ca n't explain somebod...
2        lost in a boat on the ocean lost in a ship out...
3        forgotten thing rememb the tiger eat their you...
4        oh citi shine on tomorrow for today i feel rai...
                               ...                        
19995    how can thi be ? whi is he the chosen one ? sa...
19996    i 'm a radio joe of the biz call show i 'm a r...
19997    we 've all been hurt by love it 's out of our ...
19998    i want someon to come home to i need somewher ...
19999    off the coast of africa , bound for south amer...
Name: text, Length: 20000, dtype: object

In [137]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity
tdif=TfidfVectorizer(analyzer='word', stop_words='english')

In [138]:
matrix=tdif.fit_transform(df['text'])

In [139]:
# from sklearn.metrics.pairwise import cosine_similarity
# from scipy.sparse import csr_matrix

# # Assuming 'matrix' is a sparse matrix in CSR format
# sparse_matrix = csr_matrix(matrix)

# # Compute cosine similarity using sparse matrices
similar = cosine_similarity(matrix)

In [140]:
similar[0]

array([1.        , 0.021583  , 0.00931675, ..., 0.00437166, 0.00741318,
       0.02434784])

In [141]:
df['song']

0             steel drivin' man
1        should've known better
2               hold on tightly
3                   family tree
4                    clean rain
                  ...          
19995        in the name of god
19996         punk rock classic
19997         whatever it takes
19998                 alligator
19999                lighthouse
Name: song, Length: 20000, dtype: object

In [142]:
df[df['song']=="city's burning"].index[0]

IndexError: index 0 is out of bounds for axis 0 with size 0

Now we are making recommder function for getting the songs


In [172]:
def recommder(song_name):
    idx=df[df['song']==song_name].index[0]
    distance=sorted(list(enumerate(similar[idx])), key=lambda x: x[1], reverse=True)
    song=[]
    for s_id in distance[1:20]:
        song.append(df.iloc[s_id[0]].song)
    return song
    
    

In [173]:
recommder("steel drivin' man")

['phony calls',
 'make u mine',
 'mike post theme',
 'baby featmike shorey',
 'christmas',
 'all that i want',
 'riddle of steel',
 'eyes closed',
 "it's a man's world",
 'walk like a man',
 'cats in the cradle',
 "enough's enough",
 "i'm a man",
 "'69",
 'scent of a woman',
 'back door man',
 "that's a man's way",
 "i'm waiting for the man",
 'the lord of steel']

In [184]:
pp

,song,text
artist,,
'n Sync,28,28
ABBA,44,44
Ace Of Base,29,29
Adam Sandler,25,25
Adele,20,20
...,...,...
Zoegirl,11,11
Zornik,3,3
Zox,8,8


Now we are gonna do the dumping of our code to bytes through pickle with dump method

In [203]:
def artist_songs(song_name):
    index_range=pp[pp['song']==song_name].artist
    artist_list=pp[pp['artist']==index_range.iloc[0]].song
    return artist_list

In [210]:
pde=artist_songs('phony calls').tolist()

In [211]:
pde

['toothless people',
 'gump',
 'one more minute',
 'alimony',
 'mr. frump in the iron lung',
 "frank's",
 'polkamon',
 'midnight star',
 "callin' in sick",
 'phony calls',
 "my baby's in love with eddie vedder",
 'tmz',
 'perform this way',
 'syndicated inc.',
 'george of the jungle',
 'mr. popeil',
 'my bologna',
 'traffic jam',
 'such a groovy guy',
 'fun zone',
 'another one rides the bus',
 'do i creep you out',
 'llama song',
 'talk soup',
 'smells like nirvana',
 'ricky',
 'germs',
 'cable tv',
 'jurassic park',
 'amish paradise',
 'one of those days',
 'the brady bunch',
 'melanie',
 'good enough for now']

In [ ]:
import pickle

In [ ]:
pickle.dump(similar, open('similar.pkl', 'wb'))

In [ ]:
pickle.dump(df, open('df.pkl', 'wb'))